The goal of this notebook is to run the elastic net regression. The first and second cell were used for prototyping interactively. The third cell reflects a parameter search for an ideal alpha across datasets. The resulting alphas were aggregated and averaged to select an optimal alpha, which was then used with the fourth cell to generate the data for figures in the manuscript. The fourth cell reflects a script which will take a single alpha, run it on all datasets, and output the results. 

In [1]:
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 
import os
import scipy as sc
from re import sub
from sklearn import preprocessing

#wrangling the metadata and matrix into a cohesive file
def X_Ysplit(matrixname, targetcol):
    X=pd.read_csv(sub("meta","data",matrixname), index_col=0).T
    Y=pd.read_csv(matrixname, error_bad_lines=False, index_col=0,usecols=[targetcol], sep="\t")
    Y=pd.to_numeric(Y[targetcol])
    Y=np.log2((Y+min(Y))*10000)
    X = preprocessing.StandardScaler().fit_transform(X)
    return([X,Y])

In [ ]:
os.chdir("clonality/")
todo=os.listdir()
todo=[x for x in todo if "meta" in x]
todo=[x for x in todo if "0_0" not in x]
#seting this up so that we're hitting the right target
cols=[sub(".{10,}_","", x) for x in todo]
cols=[sub("Suppression|Suppressed","bulkfreq_2", x) for x in cols]
cols=[sub("Viremic","bulkfreq_1", x) for x in cols]
cols=[sub("Uninfected|uninfected","results_freq", x) for x in cols]
#iterating through a large number of alphas to find out which works
alpha = np.logspace(-3, -1, 12)
results={}
for file, col in zip(todo,cols):
    print("starting " + file)
    cur_X, cur_Y = X_Ysplit(file, col)
    clf=linear_model.ElasticNetCV(alphas=alpha, cv=5, max_iter=10000, n_jobs=-1)
    clf.fit(cur_X, cur_Y)
    best_score = clf.score(cur_X,cur_Y)
    best_parameters = {'alpha': clf.alpha_}
    
    print("Best score: {:.2f}".format(best_score))
    print("Best parameters: {}".format(best_parameters))
    results[file]=best_parameters

In [ ]:
#helper function to quickly write results to a file
def writelist(towrite, file):
    with open(file, 'w') as filehandle:
        for listitem in towrite:
            filehandle.write('%s\n' % listitem)

writelist([results], "topalpha.txt")

In [ ]:
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 
import os
import scipy as sc
from re import sub
from sklearn import preprocessing
import argparse
parser = argparse.ArgumentParser(description="find optimal alpha and write to a file")
parser.add_argument('--input','-I',
                    help='metadata file to process')
args = parser.parse_args()

def X_Ysplit(matrixname, targetcol):
    X=pd.read_csv(sub("meta","data",matrixname), index_col=0).T
    Y=pd.read_csv(matrixname, error_bad_lines=False, index_col=0,usecols=[targetcol], sep="\t")
    Y=pd.to_numeric(Y[targetcol])
    Y=np.log2((Y+min(Y))*10000)
    X = preprocessing.StandardScaler().fit_transform(X)
    return([X,Y])

todo=[args.input]
todo=[x for x in todo if "meta" in x]
todo=[x for x in todo if "0_0" not in x]

cols=[sub(".{10,}_","", x) for x in todo]
cols=[sub("Suppression|Suppressed","bulkfreq_2", x) for x in cols]
cols=[sub("Viremic","bulkfreq_1", x) for x in cols]
cols=[sub("Uninfected|uninfected","results_freq", x) for x in cols]

alpha = np.logspace(-3, -1, 12)
results={}

for file, col in zip(todo,cols):
    print("starting " + file)
    cur_X, cur_Y = X_Ysplit(file, col)
    clf=linear_model.ElasticNetCV(alphas=alpha, cv=5, max_iter=10000, n_jobs=-1)
    clf.fit(cur_X, cur_Y)
    best_score = clf.score(cur_X,cur_Y)
    best_parameters = {'alpha': clf.alpha_}
    
    print("Best score: {:.2f}".format(best_score))
    print("Best parameters: {}".format(best_parameters))
    results[file]=best_parameters

def writelist(towrite, file):
    with open(file, 'w') as filehandle:
        for listitem in towrite:
            filehandle.write('%s\n' % listitem)

writelist([results], todo[0]+"alpha")

In [ ]:
#onealpha.py, set the alpha at the top and its just works 
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 
import os
import scipy as sc
from re import sub
from sklearn import preprocessing
import argparse
#parse args 
parser = argparse.ArgumentParser(description="find optimal alpha and write to a file")
parser.add_argument('--input','-I',
                    help='metadata file to process')
args = parser.parse_args()
#determined in the previous step
alpha = [0.0285]
#utility functions to load and save data 
def X_Ysplit(matrixname, targetcol):
    X=pd.read_csv(sub("meta","data",matrixname), index_col=0).T
    Y=pd.read_csv(matrixname, error_bad_lines=False, index_col=0,usecols=[targetcol], sep="\t")
    Y=pd.to_numeric(Y[targetcol])
    Y=np.log2((Y+min(Y))*10000)
    return([X,Y, list(X)])
#used to write gene weights 
def writecoef(model, genes,file):
    with open(file, 'w') as filehandle:
        for gene, co in zip(genes,list(model.coef_)):
            filehandle.write('%s\t%s\n' % (gene, str(co)))

#change names so that everything is teed up 

todo=[args.input]
todo=[x for x in todo if "meta" in x]
todo=[x for x in todo if "0_0" not in x]

cols=[sub(".{10,}_","", x) for x in todo]
cols=[sub("unstimulated_meta_Viremic|unstimulated_meta_Suppressed","results_freq", x) for x in cols]
cols=[sub("Suppression|Suppressed","bulkfreq_2", x) for x in cols]
cols=[sub("Viremic","bulkfreq_1", x) for x in cols]
cols=[sub("Uninfected|uninfected","results_freq", x) for x in cols]

#load data in and fit model
for file, col in zip(todo,cols):
    print("starting " + file)
    cur_X, cur_Y, X = X_Ysplit(file, col)
    cur_X = preprocessing.StandardScaler().fit_transform(cur_X)
    clf=linear_model.ElasticNetCV(alphas=alpha, cv=5, max_iter=10000, n_jobs=-1)
    clf.fit(cur_X, cur_Y)
    
writecoef(clf, list(X), todo[0]+"results")